<a href="https://colab.research.google.com/github/bforsbe/SK2534/blob/main/mutation_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mutate DNA sequence but preserve protein fold**
this exercise is meant to affirm the basics of how DNA is *translated* into proteins, and how this determines the *fold* of the protein.

### **Instructions**
The following is a set of code blocks that will set up and execute steps to help you re-design the DNA sequence of a small protein. **The aim is to attain a high mutation score**, which you will achieve by mutating DNA bases. Each mutation increases your score. If the mutation results in a mutated amino acid as well, your score will increase even more. **However, if the mutations destroy the fold of the protein, your total score will suffer**. Your total score is calculated by copying the mutated protein sequence into the ESMfold code block and predicting its structure. The following block can then calculate a similarity score in the form of RMSD, which is combined with your mutation score.

#### **What to do**
- read the instructions for each block.
- click the little ▶ play icon to the left of each block below.


First, we will install biopython, which is needed to translate DNA sequences, align structures, and more. this should be fairly quick.

In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.4 MB/s eta 0:00:00


Ok, now we can provide an original and mutated sequence, check what is different, and get a mutation score. In case you need it, the unmutated DNA sequence we are using in this exercise is

```
ATG AAA GTG ATT TTT CTG AAA AAA GGC GAA ATT AAA AAC GTG GCG GAT GGC TAT GCG AAC AAC TTT CTG TTT AAA CAG GGC CTG GCG ATT GAA GCG
```

Try altering one of the bases, and run this block to see what happened, and what score you get.


In [2]:
#@title Mutate DNA and inspect translation
from IPython.display import display, HTML
from textwrap import wrap
import Bio
from Bio.Seq import Seq

# === Inputs ===
original_seq = "ATG AAA GTG ATT TTT CTG AAA AAA GGC GAA ATT AAA AAC GTG GCG GAT GGC TAT GCG AAC AAC TTT CTG TTT AAA CAG GGC CTG GCG ATT GAA GCG" #@param {type:"string"}
mutated_seq  = "ATGCGTGTGTTATTTATTCGTAAAGGTGAATTACGTAACATTGCGGAAGCTTTTGCGCAACAATATCTGTTTCGTAATGGCTTAGCGATTGAAGCG" #@param {type:"string"}

# Clean input
original_seq = original_seq.replace(" ", "").upper()
mutated_seq  = mutated_seq.replace(" ", "").upper()

# === Checks ===
if len(original_seq) != len(mutated_seq):
    raise ValueError(f"Sequences must have the same length (got {len(original_seq)} vs {len(mutated_seq)}).")
if len(original_seq) % 3 != 0:
    raise ValueError(f"Sequence length must be a multiple of 3 (got {len(original_seq)}).")

# === Genetic code (DNA codons) ===
codon_table = {
    "TTT":"F","TTC":"F","TTA":"L","TTG":"L",
    "CTT":"L","CTC":"L","CTA":"L","CTG":"L",
    "ATT":"I","ATC":"I","ATA":"I","ATG":"M",
    "GTT":"V","GTC":"V","GTA":"V","GTG":"V",
    "TCT":"S","TCC":"S","TCA":"S","TCG":"S",
    "CCT":"P","CCC":"P","CCA":"P","CCG":"P",
    "ACT":"T","ACC":"T","ACA":"T","ACG":"T",
    "GCT":"A","GCC":"A","GCA":"A","GCG":"A",
    "TAT":"Y","TAC":"Y","TAA":"*","TAG":"*",
    "CAT":"H","CAC":"H","CAA":"Q","CAG":"Q",
    "AAT":"N","AAC":"N","AAA":"K","AAG":"K",
    "GAT":"D","GAC":"D","GAA":"E","GAG":"E",
    "TGT":"C","TGC":"C","TGA":"*","TGG":"W",
    "CGT":"R","CGC":"R","CGA":"R","CGG":"R",
    "AGT":"S","AGC":"S","AGA":"R","AGG":"R",
    "GGT":"G","GGC":"G","GGA":"G","GGG":"G"
}

def translate_dna(seq):
    codons = wrap(seq, 3)
    aa = [codon_table.get(c,"?") for c in codons]
    return codons, aa

from IPython.display import display, HTML

def compare_sequences(seq1, seq2, space_every=10, spacer=" ", no_number=False):
    """
    Display two sequences in monospace, highlight differences in bold red,
    insert spaces every n characters, and add numbering above each block.
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be the same length")

    def highlight_block(s1, s2):
        """Highlight differences in one block of equal length"""
        out1, out2 = [], []
        for a, b in zip(s1, s2):
            out1.append(a)
            if a == b:
                out2.append(b)
            else:
                out2.append(f"<span style='color:red;font-weight:bold'>{b}</span>")
        return "".join(out1), "".join(out2)

    # Split into blocks of size space_every
    blocks1 = [seq1[i:i+space_every] for i in range(0, len(seq1), space_every)]
    blocks2 = [seq2[i:i+space_every] for i in range(0, len(seq2), space_every)]

    seq1_blocks, seq2_blocks, num_blocks = [], [], []
    pos = 0
    for b1, b2 in zip(blocks1, blocks2):
        o1, o2 = highlight_block(b1, b2)
        seq1_blocks.append(o1)
        seq2_blocks.append(o2)
        pos += len(b1)
        num_blocks.append(str(pos).rjust(len(b1)))


    number_line = spacer.join(num_blocks)
    seq1_line   = spacer.join(seq1_blocks)
    seq2_line   = spacer.join(seq2_blocks)

    html = []
    if no_number:
        html = seq1_line + "<br>" + seq2_line
    else:
        html = number_line + "<br>" + seq1_line + "<br>" + seq2_line
    display(HTML(f"<pre style='font-family:monospace'>{html}</pre>"))

def dna_identity(seq1: str, seq2: str) -> float:
    """Return % identity at DNA level"""
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must have the same length")
    matches = sum(a == b for a, b in zip(seq1.upper(), seq2.upper()))
    return 100 * matches / len(seq1)

def protein_identity(seq1: str, seq2: str) -> float:
    """Translate DNA → protein and return % identity"""
    if len(seq1) % 3 != 0 or len(seq2) % 3 != 0:
        raise ValueError("DNA sequences must be divisible by 3")
    prot1 = str(Seq(seq1).translate(to_stop=False))
    prot2 = str(Seq(seq2).translate(to_stop=False))
    if len(prot1) != len(prot2):
        raise ValueError("Translated proteins have different lengths")
    matches = sum(a == b for a, b in zip(prot1, prot2))
    return 100 * matches / len(prot1)

def mutation_score(seq1: str, seq2: str) -> float:
    """Return mutation score"""
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must have the same length")

    protein_id = protein_identity(seq1, seq2)
    dna_id = dna_identity(seq1, seq2)

    return ((100.-protein_id)+(100.-dna_id))/2

# === Translation ===
# ATGCGTGTGTTATTTATTCGTCGTGGTGAATTACGTAACATTGGTGAAGCTTTTGCGCAACAATATCTGTTTCGTAATGGCTTAGCGATTGAAGCG
codons_o, aa_o = translate_dna(original_seq)
codons_m, aa_m = translate_dna(mutated_seq)

# === HTML display ===

compare_sequences(original_seq,mutated_seq,space_every=3)
compare_sequences(aa_o,aa_m,space_every=1, no_number=True, spacer="   ")

#display(HTML(f"<pre style='font-family:monospace'>{"   ".join(aa_o)}</pre>"))
#compare_sequences("".join(codons_o),"".join(codons_m),space_every=30)

print("\n\nMutaded sequence without spaces:")
display(HTML(f"<pre style='font-family:monospace'>{"".join(aa_m)}</pre>"))

dna_id = dna_identity(original_seq, mutated_seq)
#print(f"\nDNA % identity: {dna_id:.1f}%")
protein_id = protein_identity(original_seq, mutated_seq)
#print(f"Protein % identity: {protein_id:.1f}%")
print("\n")
mut_score = mutation_score(original_seq,mutated_seq)
print(f"Mutation score is {mut_score:.2f}")




Mutaded sequence without spaces:




Mutation score is 41.15


#**ESMFold**
Once we have a mutated *protein* sequence, we can fold it to make sure we didn't destroy the fold. **NOTE** if your mutation did not change the protein sequence, then your total score is the same as your mutation score (so you could try to maximize this without mutating protein at all). To fold the protein, we will be using ESMfold, a protein language model made by meta.

For more details on ESMfold see: [Github](https://github.com/facebookresearch/esm/tree/main/esm), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1)

#### **Instructions**
- Run the block to install ESMfold. This should only take a couple of minutes.
- Copy the "Mutaded sequence without spaces" from the preceding block above, into the "sequence" filed below.
- Run the prediction

**NOTE**: The prediction might take a minute the first time and be quicker when you run it subsequently. The speed might also depend on the resources you have in the session, which you can check in the upper corener of the colab notebook header.

In [ ]:
%%time
#@title install
#@markdown install ESMFold, OpenFold and download Params (~2min 30s)
version = "1"
model_name = "esmfold_v0.model" if version == "0" else "esmfold.model"
import os, time
if not os.path.isfile(model_name):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system(f"aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/{model_name} &")

  if not os.path.isfile("finished_install"):
    # install libs
    print("installing libs...")
    os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol modelcif")
    os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

    print("installing openfold...")
    # install openfold
    os.system(f"pip install -q git+https://github.com/sokrypton/openfold.git")

    print("installing esmfold...")
    # install esmfold
    os.system(f"pip install -q git+https://github.com/sokrypton/esm.git")
    os.system("touch finished_install")

  # wait for Params to finish downloading...
  while not os.path.isfile(model_name):
    time.sleep(5)
  if os.path.isfile(f"{model_name}.aria2"):
    print("downloading params...")
  while os.path.isfile(f"{model_name}.aria2"):
    time.sleep(5)

installing libs...


In [ ]:
#@title ##run **ESMFold**
%%time
from string import ascii_uppercase, ascii_lowercase
import hashlib, re, os
import numpy as np
import torch
from jax.tree_util import tree_map
import matplotlib.pyplot as plt
from scipy.special import softmax
import gc

def parse_output(output):
  pae = (output["aligned_confidence_probs"][0] * np.arange(64)).mean(-1) * 31
  plddt = output["plddt"][0,:,1]

  bins = np.append(0,np.linspace(2.3125,21.6875,63))
  sm_contacts = softmax(output["distogram_logits"],-1)[0]
  sm_contacts = sm_contacts[...,bins<8].sum(-1)
  xyz = output["positions"][-1,0,:,1]
  mask = output["atom37_atom_exists"][0,:,1] == 1
  o = {"pae":pae[mask,:][:,mask],
       "plddt":plddt[mask],
       "sm_contacts":sm_contacts[mask,:][:,mask],
       "xyz":xyz[mask]}
  return o

def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
alphabet_list = list(ascii_uppercase+ascii_lowercase)

jobname = "mutation_1" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]

sequence = "MRVLFIRKGELRNIAEAFAQQYLFRNGLAIEA" #@param {type:"string"}
sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
sequence = re.sub(":+",":",sequence)
sequence = re.sub("^[:]+","",sequence)
sequence = re.sub("[:]+$","",sequence)
copies = 1
if copies == "" or copies <= 0: copies = 1
sequence = ":".join([sequence] * copies)
num_recycles = 3
chain_linker = 25

ID = jobname+"_"+get_hash(sequence)[:5]
print(ID)
seqs = sequence.split(":")
lengths = [len(s) for s in seqs]
length = sum(lengths)
print("length",length)

u_seqs = list(set(seqs))
if len(seqs) == 1: mode = "mono"
elif len(u_seqs) == 1: mode = "homo"
else: mode = "hetero"

if "model" not in dir() or model_name != model_name_:
  if "model" in dir():
    # delete old model from memory
    del model
    gc.collect()
    if torch.cuda.is_available():
      torch.cuda.empty_cache()

  model = torch.load(model_name, weights_only=False)
  model.eval().cuda().requires_grad_(False)
  model_name_ = model_name

# optimized for Tesla T4
if length > 700:
  model.set_chunk_size(64)
else:
  model.set_chunk_size(128)

torch.cuda.empty_cache()
output = model.infer(sequence,
                     num_recycles=num_recycles,
                     chain_linker="X"*chain_linker,
                     residue_index_offset=512)

pdb_str = model.output_to_pdb(output)[0]
output = tree_map(lambda x: x.cpu().numpy(), output)
ptm = output["ptm"][0]
plddt = output["plddt"][0,...,1].mean()
O = parse_output(output)
print(f'ptm: {ptm:.3f} plddt: {plddt:.3f}')
os.system(f"mkdir -p {ID}")
prefix = f"{ID}/ptm{ptm:.3f}_r{num_recycles}_default"
np.savetxt(f"{prefix}.pae.txt",O["pae"],"%.3f")
o_pdb_file=f"{prefix}.pdb"
with open(o_pdb_file,"w") as out:
  out.write(pdb_str)

Now that we have predicted the fold of the mutated protein sequence, we will align it to a reference, and calculate their differences in atom positions, or root mean squared difference (RMSD). This will then be used to penalize proteins with a high RMSD to the reference structure, which will affect your total score. That is, if you destroyed the fold, your score will be lowered.

**NOTE**: This will be based on the last calculated mutation score and predicted model, i.e. assuming that you have run all blocks in order (you dont have to run the install blocks again though, unless you have restarted the whole session).


In [ ]:
#@title Calculate RMSD-based final score
import os, re, requests
import torch
from jax.tree_util import tree_map
from Bio.Seq import Seq
from Bio.PDB import PDBParser, Superimposer, PDBIO
import numpy as np
import tempfile

# --- Superpose prediction onto reference ---
def align_pred_to_ref(pred_pdb_str, ref_pdb_file, out_file="pred_aligned.pdb"):
    parser = PDBParser(QUIET=True)

    # Save predicted PDB string to temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdb") as tmp_pred:
        tmp_pred.write(pred_pdb_str.encode())
        pred_file = tmp_pred.name

    pred = parser.get_structure("pred", pred_file)
    ref  = parser.get_structure("ref", ref_pdb_file)

    # Use CA atoms for alignment
    pred_atoms = [a for a in pred.get_atoms() if a.get_id() == "CA"]
    ref_atoms  = [a for a in ref.get_atoms()  if a.get_id() == "CA"]

    n = min(len(pred_atoms), len(ref_atoms))
    si = Superimposer()
    si.set_atoms(ref_atoms[:n], pred_atoms[:n])  # align prediction to reference
    si.apply(pred.get_atoms())                   # apply transform to prediction

    # Write aligned prediction
    io = PDBIO()
    io.set_structure(pred)
    io.save(out_file)

    return out_file, si.rms

def rmsd_to_similarity(rmsd, sigma=2.0):
    """
    Convert RMSD (Å) to a similarity score in [0,1].
    Smaller RMSD -> score close to 1, larger RMSD -> score close to 0.

    Parameters
    ----------
    rmsd : float or np.array
        Root-mean-square deviation in angstroms.
    sigma : float
        Scaling factor controlling decay. Typical RMSDs of 1-2 Å → high similarity.

    Returns
    -------
    similarity : float or np.array
        Score in [0,1].
    """
    return np.exp(-rmsd / sigma)

# --- Download reference PDB ---
ref_file="reference.pdb"
if not os.path.isfile(ref_file):
  !wget -q -O reference.pdb https://raw.githubusercontent.com/bforsbe/SK2534/main/ref_for_mutation.pdb

aligned_file, rmsd_val = align_pred_to_ref(pdb_str, ref_file)

#for rmsd in [0.5, 1.0, 2.0, 5.0]:
#    print(rmsd, rmsd_to_similarity(rmsd))
pred_sim_score = rmsd_to_similarity(rmsd_val)
mut_score=mutation_score(original_seq,mutated_seq)

# --- Compare ---
print(f"Input file is {o_pdb_file}")
#print(f"Protein sequence: {protein_seq}")
print(f"mutation score: \t{mut_score:.2f})")
print(f"RMSD vs reference: \t{rmsd_val:.3f} Å (score: {pred_sim_score:.2f})")
print("\n\n")
print(f"Total score: \t{mut_score*pred_sim_score:.3f}")


##Submit
For fun and in the spirit of friendly competition, you can submit your score [here](https://forms.gle/TNsyQ57fn9YcAMmQA). You can submit as many as you want, and under whatever name or nickname you want. The results will be presented at the end of the exercise.

To understand what the RMSD value means, you can choose to look at the reference and prediction aligned in 3D. For RMSD values below 2, they might superpose very closely, whereas you might see two completely different folds for higher values.

In [ ]:
#@title Display predicted (superposed) + reference {run: "auto"}
import py3Dmol
from Bio.PDB import PDBParser, Superimposer, PDBIO
import tempfile, os

# --- Visualize both ---
def show_pred_and_ref_aligned(pred_pdb_str, ref_pdb_file,
                              color="pLDDT", show_sidechains=False, show_mainchains=False,
                              chains=None, vmin=50, vmax=90, size=(800,480), Ls=None, hide_pred=False):

    aligned_file, rmsd = align_pred_to_ref(pred_pdb_str, ref_pdb_file)

    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',
                        width=size[0], height=size[1])

    # Reference (fixed, green)
    with open(ref_pdb_file) as f:
        ref_pdb_str = f.read()
    view.addModel(ref_pdb_str,'pdb')
    view.setStyle({'model':0},{'cartoon':{'color':'green','opacity':0.7}})

    # Prediction (aligned, colored)
    with open(aligned_file) as f:
        pred_aligned_str = f.read()
    if not hide_pred:
        view.addModel(pred_aligned_str,'pdb')
    if color == "pLDDT":
        view.setStyle({'model':1,'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
    elif color == "rainbow":
        view.setStyle({'model':1,'cartoon': {'color':'spectrum'}})
    elif color == "chain":
        from string import ascii_uppercase
        alphabet_list = list(ascii_uppercase)
        for n,chain,c in zip(range(chains),alphabet_list,pymol_color_list):
            view.setStyle({'model':1,'chain':chain},{'cartoon': {'color':c}})

    if show_sidechains:
        view.addStyle({'model':1,'stick':{}})

    view.zoomTo()
    print(f"Structural alignment RMSD: {rmsd:.3f} Å")
    return view


# --- Params ---
color = "confidence" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = False #@param {type:"boolean"}
show_mainchains = True #@param {type:"boolean"}
hide_predicted = False #@param {type:"boolean"}

# --- Run ---

show_pred_and_ref_aligned(pdb_str, "reference.pdb",
                          color=color,
                          show_sidechains=show_sidechains,
                          show_mainchains=show_mainchains,
                          Ls=lengths,
                          hide_pred=hide_predicted).show()

The following is only useful if you want to assess the prediction, which is not the topic here, but something you can do and discuss with the instructor if you want.

In [ ]:
conservative_substitutions = {
    'A': ['G', 'S', 'T'], # Alanine (small, nonpolar)
    'R': ['K', 'H'],     # Arginine (basic, positive)
    'N': ['Q', 'S', 'T'], # Asparagine (polar, uncharged)
    'D': ['E'],          # Aspartic Acid (acidic, negative)
    'C': ['S'],          # Cysteine (sulfur-containing)
    'Q': ['N', 'H', 'K', 'R', 'E', 'D'], # Glutamine (polar, uncharged) - larger group due to flexibility
    'E': ['D', 'Q', 'N', 'K', 'R', 'H'], # Glutamic Acid (acidic, negative) - larger group due to flexibility
    'G': ['A', 'S'],     # Glycine (small, nonpolar)
    'H': ['R', 'K', 'N', 'Q'], # Histidine (basic, positive, aromatic)
    'I': ['L', 'V', 'M'], # Isoleucine (hydrophobic)
    'L': ['I', 'V', 'M'], # Leucine (hydrophobic)
    'K': ['R', 'H', 'Q', 'N'], # Lysine (basic, positive)
    'M': ['L', 'I', 'V'], # Methionine (hydrophobic, sulfur-containing)
    'F': ['Y', 'W'],     # Phenylalanine (aromatic, hydrophobic)
    'P': [],             # Proline (cyclic, often disrupts structure - few conservative substitutions)
    'S': ['T', 'N', 'Q', 'A', 'G', 'C'], # Serine (polar, uncharged, hydroxyl)
    'T': ['S', 'N', 'Q', 'A', 'G'], # Threonine (polar, uncharged, hydroxyl)
    'W': ['Y', 'F'],     # Tryptophan (aromatic, hydrophobic, large)
    'Y': ['F', 'W'],     # Tyrosine (aromatic, hydrophobic, hydroxyl)
    'V': ['I', 'L', 'M']  # Valine (hydrophobic)
}

In [ ]:
def generate_conservative_mutations(dna_sequence, codon_position):
    """
    Generates a list of mutated DNA sequences with conservative amino acid
    substitutions at a given codon position.

    Args:
        dna_sequence: The original DNA sequence string.
        codon_position: The 0-indexed position of the codon to mutate.

    Returns:
        A list of mutated DNA sequences.
    """
    start_index = codon_position * 3
    end_index = start_index + 3
    original_codon = dna_sequence[start_index:end_index]

    if original_codon not in codon_table:
        print(f"Warning: Original codon '{original_codon}' not found in codon table.")
        return []

    original_amino_acid = codon_table[original_codon]

    if original_amino_acid not in conservative_substitutions:
        # Handle stop codons or amino acids with no defined conservative substitutions
        return []

    conservative_aas = conservative_substitutions[original_amino_acid]

    mutated_sequences = []
    for sub_aa in conservative_aas:
        for codon, aa in codon_table.items():
            if aa == sub_aa:
                mutated_seq = list(dna_sequence)
                mutated_seq[start_index:end_index] = list(codon)
                mutated_sequences.append("".join(mutated_seq))

    return mutated_sequences

# Example usage (assuming original_seq is defined from previous cells)
# For testing, let's use the first codon (position 0)
# mutated_dna_list = generate_conservative_mutations(original_seq, 0)
# print(f"Original DNA: {original_seq}")
# print(f"Mutated DNA sequences (conservative substitutions at position 0):")
# for mut_seq in mutated_dna_list:
#     print(mut_seq)


# Task
Implement a genetic algorithm-like approach to optimize a protein sequence based on predicted structure and mutation scores. Start with an input DNA sequence, and iteratively propose conservative amino acid substitutions by mutating the DNA. For each proposed mutation, predict the protein structure using ESMFold, calculate a total score based on mutation score and RMSD, and accept the mutation if the total score improves. Store all tried protein sequences and report the best performing mutated DNA and protein sequences found.

## Initialize

### Subtask:
Start with the original DNA sequence as the current best sequence and calculate its initial total score. Initialize a list to store all tried protein sequences.


**Reasoning**:
Initialize the best sequence and scores, and calculate the initial total score for the original sequence. Also, initialize a list to store all tried protein sequences and add the original protein sequence to it.



In [ ]:
best_dna_sequence = original_seq
best_protein_sequence = str(Bio.Seq.Seq(best_dna_sequence).translate(to_stop=False))

best_mutation_score = mutation_score(original_seq, best_dna_sequence)

# Predict structure using ESMFold
best_output = model.infer(best_protein_sequence,
                          num_recycles=num_recycles,
                          chain_linker="X"*chain_linker,
                          residue_index_offset=512)

# Extract PDB string
best_pdb_str = model.output_to_pdb(best_output)[0]

# Align to reference and calculate RMSD
aligned_file, best_rmsd_val = align_pred_to_ref(best_pdb_str, "reference.pdb")

# Convert RMSD to similarity score
best_pred_sim_score = rmsd_to_similarity(best_rmsd_val)

# Calculate initial total score
best_total_score = best_mutation_score * best_pred_sim_score

all_tried_protein_sequences = []
all_tried_protein_sequences.append(best_protein_sequence)

print(f"Initial Best DNA Sequence: {best_dna_sequence}")
print(f"Initial Best Protein Sequence: {best_protein_sequence}")
print(f"Initial Mutation Score: {best_mutation_score:.2f}")
print(f"Initial RMSD: {best_rmsd_val:.3f}")
print(f"Initial Prediction Similarity Score: {best_pred_sim_score:.2f}")
print(f"Initial Total Score: {best_total_score:.3f}")

# Task
Iteratively mutate an input DNA sequence using conservative amino acid substitutions, translate the mutated DNA to protein, and score the protein based on a similarity score and a mutation score. Accept mutations if the total score (sum of similarity and mutation scores) increases or if the similarity score is greater than 0.9. Store and print the trial sequence and its total score for each iteration. After the optimization loop, present the best total score found and plot the mutation score, similarity score, and total score as a function of iteration for all tried sequences and separately for accepted sequences.

In [ ]:
# @title auto_seq

num_iterations = 100 #@param

best_dna_sequence = original_seq
#best_dna_sequence = "ATGCGTGTGTTATTTATTCGTCGTGGTGAATTACGTAACATTGGTGAAGCTTTTGCGCAACAATATCTGTTTCGTAATGGCTTAGCGATTGAAGCG"
best_protein_sequence = str(Bio.Seq.Seq(best_dna_sequence).translate(to_stop=False))
best_total_score = -1

accepted_mutation_scores = []
accepted_sim_scores = []
accepted_total_scores = []
accepted_iterations = []
all_mutation_scores = []
all_sim_scores = []
all_total_scores = []
all_iterations = []

all_tried_protein_sequences = []
all_tried_protein_sequences.append(best_protein_sequence)

completed_iters = 0

while completed_iters < num_iterations:
    # Choose a random codon position to mutate
    random_codon_pos = random.randint(0, num_codons - 1)

    # Generate conservative mutations for this position
    if completed_iters = 0:
        mutated_dna_list_single = best_dna_sequence
    else:
        mutated_dna_list_single = generate_conservative_mutations(best_dna_sequence, random_codon_pos)

    if not mutated_dna_list_single:
        continue # Skip to the next iteration if no mutations are possible

    # Select the first generated mutation
    single_mutated_dna_seq = mutated_dna_list_single[0]
    single_mutated_protein_seq = str(Bio.Seq.Seq(single_mutated_dna_seq).translate(to_stop=False))

    # Check if protein sequence has been tried before
    if single_mutated_protein_seq in all_tried_protein_sequences:
        continue

    all_tried_protein_sequences.append(single_mutated_protein_seq)

    # Predict structure using ESMFold
    try:
        single_mutated_output = model.infer(single_mutated_protein_seq,
                                     num_recycles=num_recycles,
                                     chain_linker="X"*chain_linker,
                                     residue_index_offset=512)

        # Extract PDB string
        single_mutated_pdb_str = model.output_to_pdb(single_mutated_output)[0]

        # Align to reference and calculate RMSD
        aligned_file_single, single_mutated_rmsd_val = align_pred_to_ref(single_mutated_pdb_str, "reference.pdb")

        # Convert RMSD to similarity score
        single_mutated_pred_sim_score = rmsd_to_similarity(single_mutated_rmsd_val)

        # Calculate mutation score
        single_current_mutation_score = mutation_score(original_seq, single_mutated_dna_seq)

        # Calculate total score
        single_current_total_score = single_current_mutation_score * single_mutated_pred_sim_score

        # Store scores for ALL tried sequences
        all_iterations.append(iteration + 1)
        all_mutation_scores.append(single_current_mutation_score)
        all_sim_scores.append(single_mutated_pred_sim_score)
        all_total_scores.append(single_current_total_score)

        # Print trial sequence and total score
        print(f"Iteration {iteration + 1}: Trial Sequence: {single_mutated_protein_seq}, Total Score: {single_current_total_score:.3f}")

        # Acceptance criteria: accept if total score improves
        if single_current_total_score > best_total_score or single_mutated_pred_sim_score > 0.9:
            best_total_score = single_current_total_score
            best_dna_sequence = single_mutated_dna_seq
            best_protein_sequence = single_mutated_protein_seq

            # Store scores for ACCEPTED sequences
            accepted_iterations.append(iteration + 1)
            accepted_mutation_scores.append(single_current_mutation_score)
            accepted_sim_scores.append(single_mutated_pred_sim_score)
            accepted_total_scores.append(single_current_total_score)

            #print(f"  Accepted mutation. New best total score: {best_total_score:.3f}")

        completed_iters += 1

    except Exception as e:
        print(f"Iteration {iteration + 1}: Error processing mutation: {e}. Skipping.")
        continue # Continue to the next iteration if an error occurs

print("\nIterative mutation process complete.")
print(f"Best Mutated DNA Sequence: {best_dna_sequence}")
print(f"Best Mutated Protein Sequence: {best_protein_sequence}")
# Recalculate final scores for the best sequence
best_mutation_score = mutation_score(original_seq, best_dna_sequence)
# To get the final RMSD and prediction similarity score for the best sequence,
# you would need to re-run the prediction and alignment for best_protein_sequence
# or store these values when the best sequence is updated.
# For simplicity, we'll just report the best total score found.
print(f"Best Total Score Found: {best_total_score:.3f}")
print(f"Total number of tried protein sequences: {len(all_tried_protein_sequences)}")
print(f"Total number of accepted mutations: {len(accepted_total_scores)}")

## Generate plots for all tried sequences

### Subtask:
After the loop, create a plot showing the mutation score, similarity score, and total score for all tried sequences as a function of iteration number.


**Reasoning**:
Create a plot showing the mutation score, similarity score, and total score for all tried sequences as a function of iteration number.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np # Import numpy

# Create a new figure and axes
plt.figure(figsize=(12, 6))

# Function to normalize data
def normalize(data):
    data = np.array(data)
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Plot the normalized scores for all tried sequences
plt.plot(all_iterations, normalize(all_mutation_scores), label='Normalized Mutation Score')
plt.plot(all_iterations, np.array(all_sim_scores), label='Normalized Similarity Score') # Convert to numpy array before multiplying and normalize
plt.plot(all_iterations, normalize(np.array(all_total_scores)), label='Normalized Total Score')   # Convert to numpy array before multiplying and normalize


# Add title and labels
plt.title('Normalized Scores for All Tried Sequences Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Normalized Score')

# Add a legend
plt.legend()

# Set y-axis limits to 0 and 1 for normalized data
plt.ylim(0, 1)

# Display the plot
plt.show()

**Reasoning**:
Create a plot showing the mutation score, similarity score, and total score for accepted sequences as a function of iteration number.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np # Import numpy

# Create a new figure and axes for the accepted sequences plot
plt.figure(figsize=(12, 6))

# Function to normalize data (re-defined for clarity in this cell)
def normalize(data):
    data = np.array(data)
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Plot the normalized scores for accepted sequences
plt.plot(accepted_iterations, normalize(accepted_mutation_scores), label='Accepted Normalized Mutation Score')
plt.plot(accepted_iterations, np.array(accepted_sim_scores), label='Accepted Normalized Similarity Score') # Convert to numpy array before multiplying and normalize
plt.plot(accepted_iterations, normalize(np.array(accepted_total_scores)), label='Accepted Normalized Total Score')   # Convert to numpy array before multiplying and normalize

# Add title and labels
plt.title('Normalized Scores for Accepted Sequences Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Normalized Score')

# Add a legend
plt.legend()

# Set y-axis limits to 0 and 1 for normalized data
plt.ylim(0, 1)

# Display the plot
plt.show()

## Summary:

### Data Analysis Key Findings

*   The optimization process successfully stored the mutation score, similarity score, and total score for every tried sequence, along with the corresponding iteration number.
*   Scores were also specifically stored for sequences that met the acceptance criteria, along with their iteration numbers.
*   The plots visualize the score progression over iterations for both all attempted mutations and only the accepted mutations.

### Insights or Next Steps

*   Analyzing the plots can provide insights into the search landscape, showing how frequently improvements are found and the typical range of scores for tried vs. accepted sequences.
*   Further analysis could involve examining the characteristics of accepted mutations (e.g., position, type of substitution) to understand what contributes to improved scores.
